In [2]:
%load_ext autoreload

# Reload all modules (except those excluded by %aimport) automatically before executing code
%autoreload 2
import re
import time
import pickle
import numpy as np

from edit_distance import SequenceMatcher
import torch
from dataset import SpeechDataset
import matplotlib.pyplot as plt

from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.model import GRUDecoder
import pickle
import argparse
import matplotlib.pyplot as plt
from neural_decoder.dataset import getDatasetLoaders
import neural_decoder.lm_utils as lmDecoderUtils
from neural_decoder.lm_utils import build_llama_1B
from neural_decoder.model import GRUDecoder
from neural_decoder.bit import BiT_Phoneme
import pickle
import argparse
from lm_utils import _cer_and_wer
import json
import os
import pandas as pd
import copy
from augmentations import GaussianSmoothing
from torch.utils.data import ConcatDataset

/home/ubuntu/miniconda/envs/speech-bci/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
np.__version__

'1.25.0'

In [4]:
def convert_sentence(s):
    s = s.lower()
    charMarks = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 "'", ' ']
    ans = []
    for i in s:
        if(i in charMarks):
            ans.append(i)
    
    return ''.join(ans)

In [5]:
base_dir = "/home/ubuntu/data/speech_5gram/lang_test"

load_lm = True
# LM decoding hyperparameters
acoustic_scale = 0.5
blank_penalty = np.log(7)

run_for_llm = True

if run_for_llm:
    return_n_best = True
    rescore = True
    nbest = 100
    print("RUNNING IN LLM MODE")
else:
    return_n_best = True
    rescore = True
    nbest = 100
    print("RUNNING IN N-GRAM MODE")
    print("RESCORE", rescore)

RUNNING IN LLM MODE


In [6]:
if load_lm and 'ngramDecoder' not in globals():
        
    lmDir = base_dir
    ngramDecoder = lmDecoderUtils.build_lm_decoder(
        lmDir,
        acoustic_scale=acoustic_scale, #1.2
        nbest=nbest,
        beam=18
    )
    print("loaded LM")

loaded LM


I0807 14:05:59.370102  7640 brain_speech_decoder.h:52] Reading fst /home/ubuntu/data/speech_5gram/lang_test/TLG.fst
I0807 14:09:13.403996  7640 brain_speech_decoder.h:58] Reading lm fst /home/ubuntu/data/speech_5gram/lang_test/G.fst
I0807 14:09:56.009298  7640 brain_speech_decoder.h:70] Reading rescore fst /home/ubuntu/data/speech_5gram/lang_test/G_no_prune.fst
I0807 14:20:18.699311  7640 brain_speech_decoder.h:81] Reading symbol table /home/ubuntu/data/speech_5gram/lang_test/words.txt


# Get model predictions
## Always check to make sure val perf matches wandb

In [7]:
output_file = 'obi'
    
device = "cuda"

time_per_sample_arr = []

log_data = True
rnn_mode = False
model_storage_path = '/home/ubuntu/data/transformer_short/'
    
models_to_run = ['transformer_short_training_fixed']

shared_output_file = ''

if len(shared_output_file) > 0:
    print("Writing to shared output file")
    write_mode = "a"
else:
    write_mode = "w"
    
seeds_list = [0,1,2,3,4,5,6,7,8,9]
partition_list = ["competition"] # "test"
run_lm = False

day_edit_distance = 0
day_seq_length = 0
prev_day = None

day_cer_dict = {}
total_wer_dict = {}

for partition in partition_list:
    
    if partition == 'train':
        saveFolder_transcripts = "/home/ubuntu/data/model_transcriptions_finetune/"
    if partition == 'test':
        saveFolder_transcripts = "/home/ubuntu/data/model_transcriptions/"
    if partition == 'competition':
        saveFolder_transcripts = "/home/ubuntu/data/model_transcriptions_comp/"
    
    print(saveFolder_transcripts)

    for seed in seeds_list:
        
        day_cer_dict[seed] = []
        total_wer_dict[seed] = []
                
        for mn, model_name_str in enumerate(models_to_run):
            
            modelPath = f"{model_storage_path}{model_name_str}_seed_{seed}"
            
            if len(shared_output_file) > 0:
                output_file = f"{shared_output_file}_seed_{seed}"
                print(output_file)
            else:
                output_file = f"{model_name_str}_seed_{seed}"
                
            print(f"Running model: {model_name_str}_seed_{seed}")
                
            with open(modelPath + "/args", "rb") as handle:
                args = pickle.load(handle)
                
            if log_data == False:
                print("USING ORIGINAL DATA, MAKE SURE THIS IS CORRECT.")
                data_file = "/home/ubuntu/data/ptDecoder_ctc"
            else:
                data_file = "/home/ubuntu/data/ptDecoder_ctc_both"
        
            trainLoaders, testLoaders, loadedData = getDatasetLoaders(
                data_file, 8
            )
            
            # if true, model is a GRU
            if 'nInputFeatures' in args.keys():
                
                if 'max_mask_pct' not in args:
                    args['max_mask_pct'] = 0
                if 'num_masks' not in args:
                    args['num_masks'] = 0
                if 'input_dropout' not in args:
                    args['input_dropout'] = 0
                if 'linderman_lab' not in args:
                    args['linderman_lab'] = False
                    
                print("Loading GRU")
                model = GRUDecoder(
                    neural_dim=args["nInputFeatures"],
                    n_classes=args["nClasses"],
                    hidden_dim=args["nUnits"],
                    layer_dim=args["nLayers"],
                    nDays=args['nDays'],
                    dropout=args["dropout"],
                    device=device,
                    strideLen=args["strideLen"],
                    kernelLen=args["kernelLen"],
                    gaussianSmoothWidth=args["gaussianSmoothWidth"],
                    bidirectional=args["bidirectional"],
                    input_dropout=args['input_dropout'], 
                    max_mask_pct=args['max_mask_pct'],
                    num_masks=args['num_masks'], 
                    linderman_lab=args['linderman_lab']
                ).to(device)

            else:
                
                if 'mask_token_zero' not in args:
                    args['mask_token_zero'] = False
                    
                # Instantiate model
                # set training relevant parameters for MEMO, doesn't matter for other runs because they are 
                # only run in eval mode.
                model = BiT_Phoneme(
                    patch_size=args['patch_size'],
                    dim=args['dim'],
                    dim_head=args['dim_head'],
                    nClasses=args['nClasses'],
                    depth=args['depth'],
                    heads=args['heads'],
                    mlp_dim_ratio=args['mlp_dim_ratio'],
                    dropout=0,
                    input_dropout=0,
                    gaussianSmoothWidth=args['gaussianSmoothWidth'],
                    T5_style_pos=args['T5_style_pos'],
                    max_mask_pct=0.0,
                    num_masks=0, 
                    mask_token_zeros=args['mask_token_zero'], 
                    num_masks_channels=0, 
                    max_mask_channels=0, 
                    dist_dict_path=0
                ).to(device)
                
                
            ckpt_path = modelPath + '/modelWeights'
            model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=True)
            model = model.to(device)
            
            
            model.eval()

            model_outputs = {
                "logits": [],
                "logitLengths": [],
                "trueSeqs": [],
                "transcriptions": [],
                'decodedSeqs': []
            }
            
            total_edit_distance = 0
            total_seq_length = 0

            if partition == "competition":

                testDayIdxs = [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20]
                    
            else:
                
                testDayIdxs = range(len(loadedData[partition])) 

            ground_truth_sentences = []
            
            #print("RESTRICTED DAYS: ", args['restricted_days'])
            
            for i, testDayIdx in enumerate(testDayIdxs):
                
                
                test_ds = SpeechDataset([loadedData[partition][i]])
                
                test_loader = torch.utils.data.DataLoader(
                    test_ds, batch_size=1, shuffle=False, num_workers=0
                )
                
                for j, (X, y, X_len, y_len, _) in enumerate(test_loader):
                            
                    X, y, X_len, y_len, dayIdx = (
                        X.to(device),
                        y.to(device),
                        X_len.to(device),
                        y_len.to(device),
                        torch.tensor([testDayIdx], dtype=torch.int64).to(device),
                    )

                    with torch.no_grad():
                        
                        pred = model.forward(X, X_len, dayIdx)
                    
                    if hasattr(model, 'compute_length'):
                        adjustedLens = model.compute_length(X_len)
                    else:
                        adjustedLens = ((X_len - model.kernelLen) / model.strideLen).to(torch.int32)
                        
                    for iterIdx in range(pred.shape[0]):
                        
                        trueSeq = np.array(y[iterIdx][0 : y_len[iterIdx]].cpu().detach())
                        model_outputs["logits"].append(pred[iterIdx].cpu().detach().numpy())
                        
                        model_outputs["logitLengths"].append(
                            adjustedLens[iterIdx].cpu().detach().item()
                        )
                        
                        model_outputs["trueSeqs"].append(trueSeq)
                        
                        decodedSeq = torch.argmax(
                            torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),
                            dim=-1,
                        ) 
                        
                        decodedSeq = torch.unique_consecutive(decodedSeq, dim=-1)
                        decodedSeq = decodedSeq.cpu().detach().numpy()
                        decodedSeq = np.array([i for i in decodedSeq if i != 0])
                        
                        matcher = SequenceMatcher(
                            a=trueSeq.tolist(), b=decodedSeq.tolist()
                        )
                        
                        model_outputs['decodedSeqs'].append(decodedSeq)
                        
                        total_edit_distance += matcher.distance()
                        total_seq_length += len(trueSeq)
                        
                        day_edit_distance += matcher.distance()
                        day_seq_length += len(trueSeq)
                        
                    transcript = loadedData[partition][i]["transcriptions"][j].strip()
                    transcript = re.sub(r"[^a-zA-Z\- \']", "", transcript)
                    transcript = transcript.replace("--", "").lower()
                    model_outputs["transcriptions"].append(transcript)
                    
                cer_day = day_edit_distance / day_seq_length
                day_cer_dict[seed].append(cer_day)
                print("CER DAY: ", cer_day)
                day_edit_distance = 0 
                day_seq_length = 0

            cer = total_edit_distance / total_seq_length
            
            print("Model performance: ", cer)

            if run_lm:
                
                print("Running n-gram LM")
                
                llm_outputs = []
                start_t = time.time()
                nbest_outputs = []
                
                for j in range(len(model_outputs["logits"])):
                    
                    logits = model_outputs["logits"][j]
                    
                    logits = np.concatenate(
                        [logits[:, 1:], logits[:, 0:1]], axis=-1
                    )  # Blank is last token
                    
                    logits = lmDecoderUtils.rearrange_speech_logits(logits[None, :, :], has_sil=True)
                    
                    nbest = lmDecoderUtils.lm_decode(
                        ngramDecoder,
                        logits[0],
                        blankPenalty=blank_penalty,
                        returnNBest=return_n_best,
                        rescore=rescore,
                    )
                    
                    nbest_outputs.append(nbest)
                    
                time_per_sample = (time.time() - start_t) / len(model_outputs["logits"])
                print(f"N-gram decoding took {time_per_sample} seconds per sample")
                time_per_sample_arr.append(time_per_sample)
                
                if run_for_llm:
                
                    print("SAVING OUTPUTS FOR LLM")
                    with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_model_outputs.pkl", "wb") as f:
                        pickle.dump(model_outputs, f)
                        
                    with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_nbest.pkl", "wb") as f:
                        pickle.dump(nbest_outputs, f)
                        
                else: 
                
                    for i in range(len(model_outputs["transcriptions"])):
                        model_outputs["transcriptions"][i] = model_outputs["transcriptions"][i].strip()
                        nbest_outputs[i] = nbest_outputs[i].strip()
                    
                    # lower case + remove puncs
                    for i in range(len(model_outputs["transcriptions"])):
                        model_outputs["transcriptions"][i] = convert_sentence(model_outputs["transcriptions"][i])

                    cer, wer = _cer_and_wer(nbest_outputs, model_outputs["transcriptions"], 
                                        outputType='speech', returnCI=True)

                    print("CER and WER after 5-gram LM: ", cer, wer)       
                    
                    out_file = os.path.join(saveFolder_transcripts, output_file)   # no extension per your spec
                    
                    with open(out_file + '.txt', write_mode, encoding="utf-8") as f:
                        f.write("\n".join(nbest_outputs)+ "\n")   # one line per LLM output  
                        
                    total_wer_dict[seed] = wer
                    
            else:
                
                with open(f"{saveFolder_transcripts}{model_name_str}_seed_{seed}_model_outputs.pkl", "wb") as f:
                        pickle.dump(model_outputs, f)
                    

/home/ubuntu/data/model_transcriptions/
Running model: transformer_short_training_fixed_seed_0


/home/ubuntu/miniconda/envs/speech-bci/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/ubuntu/transformers_with_dietcorp/src/neural_decoder/augmentations.py:170: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return self.conv(input, weight=self.weight, groups=self.groups, padding="same")
/tmp/ipykernel_7640/1875747173.py:203: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


CER DAY:  0.34675615212527966
CER DAY:  0.26091954022988506
CER DAY:  0.21884984025559107
CER DAY:  0.23390894819466249
CER DAY:  0.11300448430493273
CER DAY:  0.10357815442561205
CER DAY:  0.130558183538316
CER DAY:  0.1586452762923351
CER DAY:  0.14900398406374502
CER DAY:  0.18083333333333335
CER DAY:  0.16185476815398075
CER DAY:  0.19209558823529413
CER DAY:  0.1524163568773234
CER DAY:  0.13137032842582105
CER DAY:  0.12132024977698483
CER DAY:  0.11913357400722022
CER DAY:  0.12380952380952381
CER DAY:  0.1648568608094768
CER DAY:  0.11751662971175167
CER DAY:  0.1125370187561698
CER DAY:  0.10653266331658291
CER DAY:  0.1274601686972821
CER DAY:  0.12696941612604262
CER DAY:  0.17083333333333334
Model performance:  0.15550328092113408
Running n-gram LM
N-gram decoding took 0.2270938984372399 seconds per sample
SAVING OUTPUTS FOR LLM
Running model: transformer_short_training_fixed_seed_1
CER DAY:  0.3523489932885906
CER DAY:  0.2770114942528736
CER DAY:  0.2220447284345048
CER D